# Combining Python, SQL DBs and AWS

## Aim: Connect to your DB on AWS through python, and execute queries.  


# Using Python with MYSQL DB

In [1]:
# make sure we have the package installed
!pip install mysql-connector-python

In [2]:
## importing 'mysql.connector' 
import mysql.connector 

In [3]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "dbms"
)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost' (61)

### Documentation:

https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html

## Using a config File:

You do not want to make your credentials viewable to everyone who might see this file.  So instead of explicity stating your credentials, we want to import them from another file.  

In [3]:
import config
# from config import * don't have to write config.host, can just write host, but in this case, might have to use host

In [4]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

print(cnx) # it will print a connection object if everything is fine


In [5]:
cursor = cnx.cursor()

## SQL Commands: Data Definition Language Commands (DDL)
This section will discuss the commands through which you can define your database. The commands are as follows:

- CREATE
- DROP
- TRUNCATE
- ALTER
- BACKUP DATABASE

### CREATE

This statement is used to create a table or a database.



#### The ‘CREATE DATABASE’ Statement:
As the name suggests, this statement is used to create a database.

**Syntax**

`CREATE DATABASE DatabaseName;`

**Example**

`CREATE DATABASE Employee;`


#### The **‘CREATE TABLE’** Statement

This statement is used to create a table.

**Syntax**

`CREATE TABLE TableName (
Column1 datatype,
Column2 datatype,
Column3 datatype,
....
ColumnN datatype
);`

**Example**

`CREATE TABLE Employee_Info
(
EmployeeID int,
EmployeeName varchar(255),
Emergency ContactName varchar(255),
PhoneNumber int,
Address varchar(255),
City varchar(255),
Country varchar(255)
);`

You can also create a table using another table. Refer the below sytax and example:

#### The ‘CREATE TABLE AS’ Statement
**Syntax**

`CREATE TABLE NewTableName AS
SELECT Column1, column2,..., ColumnN
FROM ExistingTableName
WHERE ....;`

**Example**

`CREATE TABLE ExampleTable AS
SELECT EmployeeName, PhoneNumber
FROM Employee_Info;`

[MYSQL Data Types](http://www.mysqltutorial.org/mysql-data-types.aspx)

### DROP
This statement is used to drop an existing table or a database.

#### The ‘DROP DATABASE’ Statement
This statement is used to drop an existing database. When you use this statement, complete information present in the database will be lost.

**Syntax**

`DROP DATABASE DatabaseName;`

**Example**

`DROP DATABASE Employee;`

#### The ‘DROP TABLE’ Statement
This statement is used to drop an existing table. When you use this statement, complete information present in the table will be lost.

**Syntax**

`DROP TABLE TableName;`

**Example**

`DROP Table Employee_Info;`

In [ ]:
cursor.execute("DROP TABLE students_test4;")

### TRUNCATE
This command is used to delete the information present in the table but does not delete the table. So, once you use this command, your information will be lost, but not the table.

**Syntax**

`TRUNCATE TABLE TableName;`

**Example**

`TRUNCATE Table Employee_Info;`

### The ‘ALTER TABLE’ Statement
This statement is used to add, delete, modify columns in an existing table.

#### The ‘ALTER TABLE’ Statement with ADD/DROP COLUMN
You can use the ALTER TABLE statement with ADD/DROP Column command according to your need. If you wish to add a column, then you will use the ADD command, and if you wish to delete a column, then you will use the DROP COLUMN command.

**Syntax**
`ALTER TABLE TableName
ADD ColumnName Datatype;`

`ALTER TABLE TableName
DROP COLUMN ColumnName;`

**Example**

`ALTER TABLE Employee_Info
ADD BloodGroup varchar(255);`
 
`ALTER TABLE Employee_Info
DROP COLUMN BloodGroup ;`

#### The ‘ALTER TABLE’ Statement with ALTER/MODIFY COLUMN
This statement is used to change the datatype of an existing column in a table.

**Syntax**

`ALTER TABLE TableName
ALTER COLUMN ColumnName Datatype;`

**Example**
 
`ALTER TABLE Employee_Info
ADD DOB year;
ALTER TABLE Employee_Info
ALTER DOB date;`

### BACKUP DATABASE
This statement is used to create a full backup of an existing database.

**Syntax**
`BACKUP DATABASE DatabaseName
TO DISK = 'filepath';`

**Example**

`BACKUP DATABASE Employee
TO DISK = 'C:UsersSahitiDesktop';`


## SQL Commands: Constraints Used In Database
Constraints are used in a database to specify the rules for data in a table. The following are the different types of constraints:

- NOT NULL
- UNIQUE
- CHECK
- DEFAULT
- INDEX


### NOT NULL
This constraint ensures that a column cannot have a NULL value.

**Example**

 
`CREATE TABLE Employee_Info
(
EmployeeID int NOT NULL,
EmployeeName varchar(255) NOT NULL,
Emergency ContactName varchar(255),
PhoneNumber int NOT NULL,
Address varchar(255),
City varchar(255),
Country varchar(255)
);`
 
NOT NULL on ALTER TABLE
 
`ALTER TABLE Employee_Info
MODIFY PhoneNumber int NOT NULL;`

### UNIQUE
This constraint ensures that all the values in a column are unique.

**Example**

UNIQUE on Create Table
 
`CREATE TABLE Employee_Info
(
EmployeeID int NOT NULL UNIQUE,
EmployeeName varchar(255) NOT NULL,
Emergency ContactName varchar(255),
PhoneNumber int NOT NULL,
Address varchar(255),
City varchar(255),
Country varchar(255)
);`
 
UNIQUE on Multiple Columns
 
`CREATE TABLE Employee_Info
(
EmployeeID int NOT NULL,
EmployeeName varchar(255) NOT NULL,
Emergency ContactName varchar(255),
PhoneNumber int NOT NULL,
Address varchar(255),
City varchar(255),
Country varchar(255),
CONSTRAINT UC_Employee_Info UNIQUE(Employee_ID, PhoneNumber)
);`
 
UNIQUE on ALTER TABLE
 
`ALTER TABLE Employee_Info
ADD UNIQUE (Employee_ID);`
 
To drop a UNIQUE constraint
 
`ALTER TABLE  Employee_Info
DROP CONSTRAINT UC_Employee_Info;`

## Creating DB

In [5]:
from mysql.connector import errorcode

In [6]:
db_name = 'employees'

In [7]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

### Creating tables

In [10]:
DB_NAME = 'employees'

# using dictionary to save CREATE TABLE statements
TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT," # index auto created
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [11]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [12]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: already exists.
Creating table departments: already exists.
Creating table salaries: already exists.
Creating table dept_emp: already exists.
Creating table dept_manager: already exists.
Creating table titles: already exists.


## Inserting Data

In [7]:
db_name = 'employees'

In [9]:
#since we closed the connection, we need to reinstatiate it
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = db_name
)
cursor = cnx.cursor()

In [14]:
# cursor = create_cnx()

In [15]:
from datetime import date, datetime, timedelta

tomorrow = datetime.now().date() + timedelta(days=1)

add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))



In [16]:
data_employee

('Geert',
 'Vanderkelen',
 datetime.date(2019, 10, 24),
 'M',
 datetime.date(1977, 6, 14))

In [17]:
cursor

In [18]:
# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid

In [19]:
emp_no

13

In [20]:
# Insert salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

## Insert Many

cursor.executemany(operation, seq_of_params)

In [21]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [22]:
data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]
stmt = "INSERT INTO employees (first_name, hire_date) VALUES (%s, %s)"
cursor.executemany(stmt, data)

In [23]:
cnx.commit()

In [24]:
cursor.close()
cnx.close()

## Querying the DB

In [25]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [26]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = date(2005, 1, 1)
hire_end = date(2010, 12, 31)

cursor.execute(query, (hire_start, hire_end))

for (first_name, last_name, hire_date) in cursor:
  print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

cursor.close()
cnx.close()

, Jane was hired on 12 Feb 2005
, Joe was hired on 23 May 2006
, John was hired on 03 Oct 2010
, Jane was hired on 12 Feb 2005
, Joe was hired on 23 May 2006
, John was hired on 03 Oct 2010
, Jane was hired on 12 Feb 2005
, Joe was hired on 23 May 2006
, John was hired on 03 Oct 2010
, Jane was hired on 12 Feb 2005
, Joe was hired on 23 May 2006
, John was hired on 03 Oct 2010


In [86]:
cursor.close()
cnx.close()

## Applied: Working with our student data

### Import our student data

In [8]:
import json

In [9]:


f=open('students.json','r')
data=json.load(f)

### Create a connection and cursor to AWS db

In [7]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = db_name
)
cursor = cnx.cursor()

NameError: name 'db_name' is not defined

### Create a table for our student info

In [11]:

create_query = """CREATE TABLE students (
                  student_id int(7) NOT NULL UNIQUE AUTO_INCREMENT,
                  name varchar(20) NOT NULL,
                  dob date NOT NULL,
                  birthplace varchar(50) NOT NULL,
                  siblings int(2) NOT NULL,
                  years_in_nyc float(3) NOT NULL,
                  favorite_food varchar(50) NOT NULL,
                  PRIMARY KEY (student_id)
) ENGINE=innoDB
"""

In [18]:
# cursor.execute("DROP TABLE students;")

In [12]:
cursor.execute(create_query)

AttributeError: 'NoneType' object has no attribute 'fetchall'

### Insert the student data into the table

In [35]:
#
keys_list = []
for student in data:
    print(student['name'])
    keys = student.keys()
    for word in keys:
        if word not in keys_list:
            keys_list.append(word)
print (keys_list)

Sean Abu Wilson
Rebecca Chi-May Weng
Charlie Schlinkert
Clare Blessen
Dave Bletsch
Eddie Pognon
Findlay Bowditch
Gabriel Seemann
Garrett Keyes
Jake Cohen
JP Rinfret
Kenny Miyasato
Kyle McNicoll
Marco Sanchez-Ayala
Matthew R Finch
Melissa Munz
Michael Eby
Ravi Charan
Regina Ryan
Samuel Diaz
Sasha Epelbaum
Stephanie Bourdeau
Wesley Willis
Zoe Zhu
Christopher Shaw
['name', 'dob', 'birthplace', 'siblings', 'years_in_nyc', 'favorite_food']


In [ ]:
student = data[0]

In [ ]:
student

In [20]:
print(data)

[{'name': 'Sean Abu Wilson', 'dob': '1985-02-06', 'birthplace': 'Birmingham, AL', 'siblings': 2, 'years_in_nyc': 8.4, 'favorite_food': 'oatmeal raisin cookies'}, {'name': 'Rebecca Chi-May Weng', 'dob': '1996-05-17', 'birthplace': 'New York, NY', 'siblings': 0, 'years_in_nyc': 18, 'favorite_food': 'Taiwanese beef noodle soup'}, {'name': 'Charlie Schlinkert', 'dob': '1994-06-11', 'birthplace': 'Darien, Ct', 'siblings': 2, 'years_in_nyc': 7.2, 'favorite_food': 'Hamburgers'}, {'name': 'Clare Blessen', 'dob': '1994-08-10', 'birthplace': 'Chicago, IL', 'siblings': 1, 'years_in_nyc': 2, 'favorite_food': 'cheese'}, {'name': 'Dave Bletsch', 'dob': '1982-04-29', 'birthplace': 'New Hyde Park, NY', 'siblings': 2, 'years_in_nyc': 0.6, 'favorite_food': 'buffalo wings'}, {'name': 'Eddie Pognon', 'dob': '1994-04-26', 'birthplace': 'Newark, New Jersey', 'siblings': 4, 'years_in_nyc': 22, 'favorite_food': 'pasta'}, {'name': 'Findlay Bowditch', 'dob': '1992-08-22', 'birthplace': 'White Plains, NY', 'sibl

In [14]:
students_list = []
for student in data:
    student_i = (student['name'], student['dob'], student['birthplace'], 
                 student['siblings'], student['years_in_nyc'], student['favorite_food'])
    students_list.append(student_i)
print (students_list)

[('Sean Abu Wilson', '1985-02-06', 'Birmingham, AL', 2, 8.4, 'oatmeal raisin cookies'), ('Rebecca Chi-May Weng', '1996-05-17', 'New York, NY', 0, 18, 'Taiwanese beef noodle soup'), ('Charlie Schlinkert', '1994-06-11', 'Darien, Ct', 2, 7.2, 'Hamburgers'), ('Clare Blessen', '1994-08-10', 'Chicago, IL', 1, 2, 'cheese'), ('Dave Bletsch', '1982-04-29', 'New Hyde Park, NY', 2, 0.6, 'buffalo wings'), ('Eddie Pognon', '1994-04-26', 'Newark, New Jersey', 4, 22, 'pasta'), ('Findlay Bowditch', '1992-08-22', 'White Plains, NY', 2, 1, 'BBQ'), ('Gabriel Seemann', '1991-02-09', 'San Diego, CA', 3, 2.2, 'Falafel'), ('Garrett Keyes', '1993-09-20', 'New York City, NY', 0, 3.3, 'lamb vindaloo'), ('Jake Cohen', '1995-02-12', 'Philadelphia, PA', 1, 1.4, 'pizza'), ('JP Rinfret', '1990-08-02', 'Manhasset, NY', 2, 5.6, 'breaded chicken cutlets'), ('Kenny Miyasato', '1988-07-13', 'Lima, Peru', 0, 0, 'Pizza'), ('Kyle McNicoll', '1994-02-11', 'NYC', 1, 24, 'salmon sushi'), ('Marco Sanchez-Ayala', '1996-04-11', '

In [15]:
insert_statement = "INSERT INTO students (name, dob, birthplace, siblings, years_in_nyc, favorite_food) VALUES (%s, %s, %s, %s, %s, %s)"
cursor.executemany(insert_statement, students_list)
cnx.commit()

In [16]:
query = ("SELECT * FROM students")
cursor.execute(query)

In [17]:
# for x in cursor:
#     print(x)

for (student_id, name, dob, birthplace, siblings, years_in_nyc, favorite_food) in cursor:
    print("{} was born on {:%d %b %Y} in {}. They have {} siblings, and have lived in New York for {}. Their favorite food is {}.".format(
        name, dob, birthplace, siblings, years_in_nyc, favorite_food))

Sean Abu Wilson was born on 06 Feb 1985 in Birmingham, AL. They have 2 siblings, and have lived in New York for 8.4. Their favorite food is oatmeal raisin cookies.
Rebecca Chi-May Weng was born on 17 May 1996 in New York, NY. They have 0 siblings, and have lived in New York for 18.0. Their favorite food is Taiwanese beef noodle soup.
Charlie Schlinkert was born on 11 Jun 1994 in Darien, Ct. They have 2 siblings, and have lived in New York for 7.2. Their favorite food is Hamburgers.
Clare Blessen was born on 10 Aug 1994 in Chicago, IL. They have 1 siblings, and have lived in New York for 2.0. Their favorite food is cheese.
Dave Bletsch was born on 29 Apr 1982 in New Hyde Park, NY. They have 2 siblings, and have lived in New York for 0.6. Their favorite food is buffalo wings.
Eddie Pognon was born on 26 Apr 1994 in Newark, New Jersey. They have 4 siblings, and have lived in New York for 22.0. Their favorite food is pasta.
Findlay Bowditch was born on 22 Aug 1992 in White Plains, NY. They

In [18]:
cursor.close()
cnx.close()

### Write queries to answer the following questions:

**Questions**
- Which student is the youngest?
- Which student's birthday is closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- What are the most popular foods among the class?

In [8]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = db_name
)
cursor = cnx.cursor()

[Utilizing the date data type in MySQL](http://www.mysqltutorial.org/mysql-date/)

In [39]:
cursor.execute("""SELECT 
                     *,
                     RANK() OVER (ORDER BY dob DESC) AS birth_rank
                  FROM students;
                  """)
cursor.fetchone()

(21,
 'Sasha Epelbaum',
 datetime.date(1996, 10, 5),
 'NYC, NY',
 1,
 19.0,
 'nyc bagels',
 1)

In [50]:
cursor.execute("""SELECT
                    *,
                    RANK() OVER (ORDER BY ABS(DAYOFYEAR(dob) - DAYOFYEAR('2020-01-24')) ASC) AS date_diff
                  FROM students;
""")
cursor.fetchone()

(1,
 'Sean Abu Wilson',
 datetime.date(1985, 2, 6),
 'Birmingham, AL',
 2,
 8.4,
 'oatmeal raisin cookies',
 1)

In [30]:
cursor.execute("""SELECT
                    *
                  FROM students
                  WHERE
                    siblings = (SELECT MAX(siblings) FROM students);
""")
cursor.fetchall()

[(6,
  'Eddie Pognon',
  datetime.date(1994, 4, 26),
  'Newark, New Jersey',
  4,
  22.0,
  'pasta'),
 (23,
  'Wesley Willis',
  datetime.date(1992, 6, 4),
  'Detroit, MI',
  4,
  7.0,
  'banana fritters')]

In [38]:
cursor.execute("""SELECT
                    siblings,
                    COUNT(siblings)
                  FROM students
                  GROUP BY siblings
                  HAVING siblings = 0;
""")
cursor.fetchall()

[(0, 5)]

In [41]:
cursor.execute("""SELECT
                    *,
                    RANK() OVER (ORDER BY years_in_nyc ASC) AS years
                  FROM students
                  LIMIT 3;
""")
cursor.fetchall()

[(12,
  'Kenny Miyasato',
  datetime.date(1988, 7, 13),
  'Lima, Peru',
  0,
  0.0,
  'Pizza',
  1),
 (5,
  'Dave Bletsch',
  datetime.date(1982, 4, 29),
  'New Hyde Park, NY',
  2,
  0.6,
  'buffalo wings',
  2),
 (7,
  'Findlay Bowditch',
  datetime.date(1992, 8, 22),
  'White Plains, NY',
  2,
  1.0,
  'BBQ',
  3)]

In [42]:
cursor.execute("""SELECT
                    *
                  FROM students
                  WHERE
                    SUBSTRING_INDEX(birthplace, ', ', -1) = 'NY'
                  OR
                    birthplace = 'NYC';
""")
cursor.fetchall()

[(2,
  'Rebecca Chi-May Weng',
  datetime.date(1996, 5, 17),
  'New York, NY',
  0,
  18.0,
  'Taiwanese beef noodle soup'),
 (5,
  'Dave Bletsch',
  datetime.date(1982, 4, 29),
  'New Hyde Park, NY',
  2,
  0.6,
  'buffalo wings'),
 (7,
  'Findlay Bowditch',
  datetime.date(1992, 8, 22),
  'White Plains, NY',
  2,
  1.0,
  'BBQ'),
 (9,
  'Garrett Keyes',
  datetime.date(1993, 9, 20),
  'New York City, NY',
  0,
  3.3,
  'lamb vindaloo'),
 (11,
  'JP Rinfret',
  datetime.date(1990, 8, 2),
  'Manhasset, NY',
  2,
  5.6,
  'breaded chicken cutlets'),
 (13,
  'Kyle McNicoll',
  datetime.date(1994, 2, 11),
  'NYC',
  1,
  24.0,
  'salmon sushi'),
 (20,
  'Samuel Diaz',
  datetime.date(1987, 9, 21),
  'Harlem, NY',
  2,
  32.0,
  'Grilled Steak with Rice and Broccoli'),
 (21,
  'Sasha Epelbaum',
  datetime.date(1996, 10, 5),
  'NYC, NY',
  1,
  19.0,
  'nyc bagels'),
 (22,
  'Stephanie Bourdeau',
  datetime.date(1993, 9, 30),
  'Brooklyn, NY',
  2,
  26.1,
  'NYC pizza')]

In [13]:
cursor.execute("""SELECT
                    favorite_food,
                    COUNT(favorite_food), 
                    RANK() OVER (ORDER BY COUNT(favorite_food) DESC) AS food_rank
                  FROM students
                  GROUP BY
                    favorite_food;
""")
cursor.fetchone()

('pizza', 2, 1)